# Requirements

In [1]:
import numpy as np
import pandas as pd 
import polars as pl
from sklearn.linear_model import LinearRegression

# Functions

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data\ts-forecasting.zip
../data\ts-forecasting\test.parquet
../data\ts-forecasting\train.parquet


In [3]:
def _clip01(x: float) -> float:
    return float(np.minimum(np.maximum(x, 0.0), 1.0))

def weighted_rmse_score(y_target, y_pred, w) -> float:
    denom = np.sum(w * y_target ** 2)
    ratio = np.sum(w * (y_target - y_pred) ** 2) / denom
    clipped = _clip01(ratio)
    val = 1.0 - clipped
    return float(np.sqrt(val))

def predict_linear(X, params):
    w = params[:-1]
    b = params[-1]
    return X @ w + b

# Load data

In [4]:
df = pl.read_parquet('../data/ts-forecasting/train.parquet')
df.head()

id,code,sub_code,sub_category,horizon,ts_index,feature_a,feature_b,feature_c,feature_d,feature_e,feature_f,feature_g,feature_h,feature_i,feature_j,feature_k,feature_l,feature_m,feature_n,feature_o,feature_p,feature_q,feature_r,feature_s,feature_t,feature_u,feature_v,feature_w,feature_x,feature_y,feature_z,feature_aa,feature_ab,feature_ac,feature_ad,feature_ae,…,feature_az,feature_ba,feature_bb,feature_bc,feature_bd,feature_be,feature_bf,feature_bg,feature_bh,feature_bi,feature_bj,feature_bk,feature_bl,feature_bm,feature_bn,feature_bo,feature_bp,feature_bq,feature_br,feature_bs,feature_bt,feature_bu,feature_bv,feature_bw,feature_bx,feature_by,feature_bz,feature_ca,feature_cb,feature_cc,feature_cd,feature_ce,feature_cf,feature_cg,feature_ch,y_target,weight
str,str,str,str,i32,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64
"""W2MW3G2L__J0G2B0KU__PZ9S1Z4V__…","""W2MW3G2L""","""J0G2B0KU""","""PZ9S1Z4V""",25,89,29,16.364093,7.464023,5.966933,1.622184,10.26136,4.914369,0.000467,0.023686,0.006409,0.000187,0.744244,2.001013,-0.01687,0.009892,0.013162,0.021502,0.901966,0.402125,0.038566,0.177947,0.091141,-84.968733,-1.765306,10.109641,145.320404,0.08958,0.868698,0.080088,0.101631,0.026555,…,0.01707,0.709292,21.80395,0.120968,26999.430482,34126.269444,791.709562,0.15467,9499.742248,1.266071,429.318704,2540.88981,0.008927,1.122459,23.815924,0.54985,0.067941,0.076033,0.02759,-0.47269,-0.202944,-3.769914,0.104535,3.040304,4.499546,null,-0.058543,-0.001686,-0.105328,-0.005045,null,-0.133697,2.849819,0.112068,1,-0.551324,40.982572
"""W2MW3G2L__J0G2B0KU__PZ9S1Z4V__…","""W2MW3G2L""","""J0G2B0KU""","""PZ9S1Z4V""",1,89,53,2.858806,5.050617,15.906651,10.879453,3.072151,4.091032,0.000467,0.023686,0.006409,0.000187,0.744244,2.001013,-0.01687,0.009892,0.013162,0.021502,0.901966,0.402125,0.038566,0.177947,0.091141,-84.968733,-1.765306,10.109641,145.320404,0.08958,0.868698,0.080088,0.101631,0.026555,…,0.01707,0.709292,21.80395,0.120968,26999.430482,34126.269444,791.709562,0.15467,9499.742248,1.266071,429.318704,2540.88981,0.008927,1.122459,23.815924,0.54985,0.067941,0.076033,0.02759,-0.47269,-0.202944,-3.769914,0.104535,3.040304,4.499546,null,-0.058543,-0.001686,-0.105328,-0.005045,null,-0.133697,2.849819,0.112068,1,-0.315583,150.075406
"""W2MW3G2L__J0G2B0KU__PZ9S1Z4V__…","""W2MW3G2L""","""J0G2B0KU""","""PZ9S1Z4V""",3,89,51,9.585452,1.076268,9.004147,16.74049,15.166901,11.427983,0.000467,0.023686,0.006409,0.000187,0.744244,2.001013,-0.01687,0.009892,0.013162,0.021502,0.901966,0.402125,0.038566,0.177947,0.091141,-84.968733,-1.765306,10.109641,145.320404,0.08958,0.868698,0.080088,0.101631,0.026555,…,0.01707,0.709292,21.80395,0.120968,26999.430482,34126.269444,791.709562,0.15467,9499.742248,1.266071,429.318704,2540.88981,0.008927,1.122459,23.815924,0.54985,0.067941,0.076033,0.02759,-0.47269,-0.202944,-3.769914,0.104535,3.040304,4.499546,null,-0.058543,-0.001686,-0.105328,-0.005045,null,-0.133697,2.849819,0.112068,1,-0.362894,115.953552
"""W2MW3G2L__J0G2B0KU__PZ9S1Z4V__…","""W2MW3G2L""","""J0G2B0KU""","""PZ9S1Z4V""",10,89,44,8.840588,15.034634,4.17078,1.584433,5.383462,3.099428,0.000467,0.023686,0.006409,0.000187,0.744244,2.001013,-0.01687,0.009892,0.013162,0.021502,0.901966,0.402125,0.038566,0.177947,0.091141,-84.968733,-1.765306,10.109641,145.320404,0.08958,0.868698,0.080088,0.101631,0.026555,…,0.01707,0.709292,21.80395,0.120968,26999.430482,34126.269444,791.709562,0.15467,9499.742248,1.266071,429.318704,2540.88981,0.008927,1.122459,23.815924,0.54985,0.067941,0.076033,0.02759,-0.47269,-0.202944,-3.769914,0.104535,3.040304,4.499546,null,-0.058543,-0.001686,-0.105328,-0.005045,null,-0.133697,2.849819,0.112068,1,-0.667023,64.573073
"""W2MW3G2L__J0G2B0KU__PZ9S1Z4V__…","""W2MW3G2L""","""J0G2B0KU""","""PZ9S1Z4V""",25,90,28,2.303825,7

In [5]:
cols = ['feature_a','feature_b','feature_c','feature_d','feature_e','feature_f','feature_g','feature_h','feature_i','feature_j','feature_k',
 'feature_l','feature_m','feature_n','feature_o','feature_p','feature_q','feature_r','feature_s','feature_t','feature_u','feature_v','feature_w',
 'feature_x','feature_y','feature_z','feature_aa','feature_ab','feature_ac','feature_ad','feature_ae','feature_af','feature_ag','feature_ah','feature_ai',
 'feature_aj','feature_ak','feature_al','feature_am','feature_an','feature_ao','feature_ap','feature_aq','feature_ar','feature_as','feature_at',
 'feature_au','feature_av','feature_aw','feature_ax','feature_ay','feature_az','feature_ba','feature_bb','feature_bc','feature_bd','feature_be',
 'feature_bf','feature_bg','feature_bh','feature_bi','feature_bj','feature_bk','feature_bl','feature_bm','feature_bn','feature_bo','feature_bp',
 'feature_bq','feature_br','feature_bs','feature_bt','feature_bu','feature_bv','feature_bw','feature_bx','feature_by','feature_bz','feature_ca',
 'feature_cb','feature_cc','feature_cd','feature_ce','feature_cf','feature_cg','feature_ch']

# Basic example

In [6]:
X = df.select(cols).fill_null(0).to_numpy()
y = df.select("y_target").to_numpy().ravel()
w = df.select("weight").to_numpy().ravel()

model = LinearRegression(fit_intercept=True)
model.fit(X, y, sample_weight=w)

y_pred = model.predict(X)

score = weighted_rmse_score(y, y_pred, w)

print("Score :", score)

Score : 0.06438822149586454


# DataLeakage

## Example 1: Using y_target for normalization



In [7]:
df = df.sort('ts_index')
df = df.with_columns((pl.col("y_target") /
                      pl.col("y_target").std().over(
                          ["code", "sub_code", "sub_category", "horizon"])).alias("y_target_norm"))
df = df.drop_nulls(subset=["y_target_norm", "weight"])
X = df.select(cols).fill_null(0).to_numpy()
y = df.select("y_target_norm").to_numpy().ravel()
w = df.select("weight").to_numpy().ravel()

model = LinearRegression(fit_intercept=True)
model.fit(X, y, sample_weight=w)
y_pred = model.predict(X)
score = weighted_rmse_score(y, y_pred, w)

print("Example 1 Score:", score)

Example 1 Score: 0.13167843081727032


Here, the target variable is normalized using a standard deviation computed over all observations within each (code, sub_code, sub_category, horizon) group. Since this statistic is calculated across the entire time span of the group, it includes future values relative to each individual timestamp.

## Example 2: Using an average over the entire period

In [8]:
df = df.sort('ts_index')

cols2 = ['feature_z','feature_aa','feature_ab','feature_ac','feature_ad','feature_ae','feature_af','feature_ag','feature_ah','feature_ai',
 'feature_aj','feature_ak','feature_al','feature_am','feature_an','feature_ao','feature_ap','feature_aq','feature_ar','feature_as','feature_at',
 'feature_au','feature_av','feature_aw']
df = df.with_columns([
    pl.col(c).mean().alias(f"{c}_mean") for c in cols2
])
X = df.select(cols + [f"{c}_mean" for c in cols2]).fill_null(0).to_numpy()
y = df.select("y_target").to_numpy().ravel()
w = df.select("weight").to_numpy().ravel()

model = LinearRegression(fit_intercept=True)
model.fit(X, y, sample_weight=w)
y_pred = model.predict(X)
score = weighted_rmse_score(y, y_pred, w)

print("Example 2 Score:", score)

Example 2 Score: 0.06449361518318206


In this configuration, the mean of each feature is computed over the entire dataset, i.e. using all available timestamps, including future observations relative to each row.As a result, each row is augmented with statistics that implicitly contain information from the future, which would not be available at inference time in a production setting.

## Example 3: Using a feature with data shifted into the future

In [9]:
df = df.sort('ts_index')

cols2 = ['feature_z','feature_aa','feature_ab','feature_ac','feature_ad','feature_ae','feature_af','feature_ag','feature_ah','feature_ai',
 'feature_aj','feature_ak','feature_al','feature_am','feature_an','feature_ao','feature_ap','feature_aq','feature_ar','feature_as','feature_at',
 'feature_au','feature_av','feature_aw']
df = df.with_columns([
    pl.col(c).shift(-50).alias(f"{c}_lag50") for c in cols2
])
X = df.select(cols + [f"{c}_lag50" for c in cols2]).fill_null(0).to_numpy()
y = df.select("y_target").to_numpy().ravel()
w = df.select("weight").to_numpy().ravel()

model = LinearRegression(fit_intercept=True)
model.fit(X, y, sample_weight=w)
y_pred = model.predict(X)
score = weighted_rmse_score(y, y_pred, w)

print("Example 3 Score:", score)

Example 3 Score: 0.0649450444317564


In this configuration, lagged features are constructed using future observations (e.g. via negative shifts or centered windows), so each row incorporates information from timestamps t+1,t+2
This causes data leakage, since the model has access to future values that would not be available at inference time, resulting in overly optimistic evaluation metrics.

# Conclusion

In all these configurations, the transformation is calculated based on statistics calculated over the entire period. Consequently, the model indirectly uses future information that will not be available at the time of inference in production. The same problem arises when a feature is constructed from window data including ts_index values greater than the ts_index of the row, rather than being limited to past observations alone. This creates data leakage, leading to overly optimistic evaluation metrics and a model that is unlikely to generalize once deployed.  Similarly, implementing a shift in y_target is not feasible, as this could lead to data leakage, given that you do not have sufficient information about its construction.